模型文件地址
    
    https://github.com/PaddlePaddle/models/tree/develop/PaddleCV/image_classification
   

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import time
import sys
import math
import numpy as np
import argparse
import functools

import paddle
import paddle.fluid as fluid

from sdk.resnet import ResNet50


In [2]:
#http://www.paddlepaddle.org.cn/documentation/docs/zh/1.5/user_guides/howto/prepare_data/index_cn.html

mean = [0.485, 0.456, 0.406] 
std = [0.229, 0.224, 0.225] 

class_dim=1000
image_shape=[3,224,224]
model_name="resnet50"
use_gpu=False
pretrained_model="ResNet50_pretrained"
TOPK=3

image = fluid.layers.data(name='image', shape=image_shape, dtype='float32')

# model definition
model = ResNet50()
out = model.net(input=image, class_dim=class_dim)
out = fluid.layers.softmax(out)

test_program = fluid.default_main_program().clone(for_test=True)

fetch_list = [out.name]

place = fluid.CUDAPlace(0) if use_gpu else fluid.CPUPlace()
exe = fluid.Executor(place)
exe.run(fluid.default_startup_program())

fluid.io.load_persistables(exe, pretrained_model)



In [3]:
img_path = "mug227.png"
img = cv2.imread(img_path)
img = cv2.resize(img,(image_shape[1],image_shape[2]))

img = img[:, :, ::-1].astype('float32').transpose((2, 0, 1)) / 255
img_mean = np.array(mean).reshape((3, 1, 1))
img_std = np.array(std).reshape((3, 1, 1))
img -= img_mean
img /= img_std

print(img.shape)

result = exe.run(test_program,
                 fetch_list=fetch_list,
                 feed={ 'image':(img,) })
result = result[0][0]

pred_label = np.argsort(result)[::-1][:TOPK]
print("Test-score: {0}, class {1}".format(result[pred_label], pred_label))
sys.stdout.flush()

(3, 224, 224)
Test-score: [0.63326067 0.23805845 0.0749701 ], class [504 967 968]
